In [ ]:
from tabula import read_pdf
import pandas as pd
import os

# Path to your PDF
pdf_path = '../../../Academic_Papers/senate_insurance_report_tables.pdf'

table_ranges = [
    (1, 2),    # Table 1: pages 1-2
    (3, 4),    # Table 2: pages 3-4
    (5, 6),    # Table 3: pages 5-6
    (7, 8),    # Table 4: pages 7-8
    (9, 9),    # Table 5: page 9
    (10, 10),  # Table 6: page 10
    (11, 11),  # Table 7: page 11
]

table_1_cols = ['index', 'county', 'state', 'non_renewal_pct_2018', 'non_renewal_pct_2023', 'annual_premium_2023', 'premium_change_2018_2023']
table_2_cols = ['index', 'county', 'state', 'non_renewal_change_2018_2023', 'non_renewal_pct_2018', 'non_renewal_pct_2023', 'premium_change_2018_2023']
table_3_cols = table_1_cols
table_4_cols = table_2_cols
table_5_cols = ['index', 'state', 'non_renewal_pct_2018', 'non_renewal_pct_2023', 'non_renewal_change_2018_2023']
table_6_cols = table_5_cols
table_7_cols = ['index', 'state', 'non_renewal_pct_2018', 'non_renewal_pct_2023', 'non_renewal_pct_change_2018_2023']

column_sets = [
    table_1_cols,
    table_2_cols,
    table_3_cols,
    table_4_cols,
    table_5_cols,
    table_6_cols,
    table_7_cols
]

for i, (start_page, end_page) in enumerate(table_ranges):
    print(f"\nProcessing table {i + 1} (pages {start_page}-{end_page})")

    # Extract tables from the PDF
    tables = read_pdf(
        pdf_path,
        pages=[start_page, end_page],  # Specify page range
        multiple_tables=False,  # Treat as single table
        lattice=False,
        stream=True
    )

    if tables:
        # Combine tables if multi-page
        table = pd.concat(tables, ignore_index=True) if len(tables) > 1 else tables[0]
        
        # Use first row as column names and drop it from data
        table = table.iloc[1:].reset_index(drop=True)
        
        # Assign correct column names
        table.columns = column_sets[i]
        
        # Clean up numeric columns
        numeric_columns = [col for col in table.columns if 'pct' in col or 'premium' in col or 'change' in col]
        for col in numeric_columns:
            table[col] = pd.to_numeric(table[col], errors='coerce')
        
        # Save to CSV
        csv_path = f"../data/home_insurance/table_{i + 1}.csv"
        table.to_csv(csv_path, index=False)
        print(f"Saved table {i + 1} to {csv_path}")
    else:
        print(f"No table found in pages {start_page}-{end_page}")

In [11]:
counties_10k = pd.read_csv("../data/home_insurance/table_1.csv")
counties_1k = pd.read_csv("../data/home_insurance/table_3.csv")

census_home_values = pd.read_csv("../data/home_insurance/county_home_values.csv")

print(counties_10k.head())
print(census_home_values.head())

   index      county state  non_renewal_pct_2018  non_renewal_pct_2023  \
0    1.0        LAKE    CA                  1.24                  7.56   
1    2.0      NEVADA    CA                  2.30                  6.51   
2    3.0  BARNSTABLE    MA                  0.78                  6.39   
3    4.0    TUOLUMNE    CA                  7.33                  6.10   
4    5.0     JACKSON    MS                  0.32                  5.55   

   annual_premium_2023  premium_change_2018_2023  
0               2707.0                    1041.0  
1               3868.0                    1888.0  
2               3057.0                     880.0  
3                  NaN                       NaN  
4               4265.0                    1395.0  
   county_fips       county_name  state_fips      state_name  home_value_2023  \
0            1  Abbeville County          45  South Carolina         147200.0   
1            1     Acadia Parish          22       Louisiana         146900.0   
2     

In [16]:
# First, create a state abbreviation to name dictionary
state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Clean and prepare the census data
census_df = census_home_values.copy()
census_df['county_name_clean'] = (census_df['county_name']
                                 .str.split()
                                 .str[:-1]
                                 .str.join('')
                                 .str.replace('.', '')
                                 .str.upper())

# Clean and prepare the senate data
senate_df = counties_10k.copy()
senate_df['county_clean'] = (senate_df['county']
                            .str.replace('.', '')
                            .str.replace(' ', ''))
senate_df['state_name'] = senate_df['state'].map(state_abbrev)

# Merge the dataframes
merged_df = pd.merge(
    senate_df,
    census_df,
    left_on=['county_clean', 'state_name'],
    right_on=['county_name_clean', 'state_name'],
    how='left'
)

# Clean up the merged dataframe
merged_df = merged_df.drop(['county_name_clean', 'county_clean'], axis=1)

# Check for unmatched counties
unmatched = merged_df[merged_df['county_fips'].isna()]
print("\nUnmatched counties:")
print(unmatched[['county', 'state', 'state_name']])

# Print matching statistics
total_counties = len(senate_df)
matched_counties = len(merged_df) - len(unmatched)
print(f"\nMatching Statistics:")
print(f"Total counties in senate data: {total_counties}")
print(f"Successfully matched: {matched_counties}")
print(f"Match rate: {(matched_counties/total_counties)*100:.1f}%")

# Preview the merged data
print("\nMerged Data Preview:")
print(merged_df.head())
merged_df.to_csv("../data/home_insurance/senate_10k_census_merged.csv", index=False)


Unmatched counties:
Empty DataFrame
Columns: [county, state, state_name]
Index: []

Matching Statistics:
Total counties in senate data: 100
Successfully matched: 100
Match rate: 100.0%

Merged Data Preview:
   index      county state  non_renewal_pct_2018  non_renewal_pct_2023  \
0    1.0        LAKE    CA                  1.24                  7.56   
1    2.0      NEVADA    CA                  2.30                  6.51   
2    3.0  BARNSTABLE    MA                  0.78                  6.39   
3    4.0    TUOLUMNE    CA                  7.33                  6.10   
4    5.0     JACKSON    MS                  0.32                  5.55   

   annual_premium_2023  premium_change_2018_2023     state_name  county_fips  \
0               2707.0                    1041.0     California           33   
1               3868.0                    1888.0     California           57   
2               3057.0                     880.0  Massachusetts            1   
3                  NaN     